# Use Metrics

This tutorial is still WIP.

In [ ]:
import argilla as rg
from argilla._constants import DEFAULT_API_KEY

In [ ]:
# Argilla credentials
api_url = "http://localhost:6900"  # "https://<YOUR-HF-SPACE>.hf.space"
api_key = DEFAULT_API_KEY  # admin.apikey
# Huggingface credentials
hf_token = "hf_..."